In [74]:
import numpy
import scipy.special
import matplotlib.pyplot

class NumbersRecognition:
    # initialise the neural network
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        
        # set numbers of nodes in each input, hidden and output layer
        self.i_nodes = input_nodes
        self.h_nodes = hidden_nodes
        self.o_nodes = output_nodes
        
        # link weight matrices, w_ih and w_ho
        
        # w_ih is a matrix (h_nodes, i_nodes), where number of rows = h_nodes, and number of columns = i_nodes.
        # ==> in current test the matrix is (100, 784), where number of rows = 100 and number of columns = 784.
        self.w_ih = numpy.random.normal(0.0, pow(self.h_nodes, -0.5), (self.h_nodes, self.i_nodes))
        
        # w_ho is a matrix (o_nodes, h_nodes), where number of rows = o_nodes, and number of columns = h_nodes.
        # ==> in current test the matrix is (10, 100), where number of rows = 10 and number of columns = 100.
        self.w_ho = numpy.random.normal(0.0, pow(self.o_nodes, -0.5), (self.o_nodes, self.h_nodes))
        
        # learning rate
        self.lr = learning_rate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        pass

    # train the neural network
    def train(self, inputs_list, targets_list):
        
        # convert inputs list to 2-D array
        # .T is transpose of this matrix
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate input and output signals for hidden layer
        # numpy.dot returns the dot product of two arrays. For 2-D vectors, it is the equivalent to matrix multiplication.
        hidden_inputs = numpy.dot(self.w_ih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate input and output signals for output layer
        final_inputs = numpy.dot(self.w_ho, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        # output layer errors = targets value - actual value
        output_errors = targets - final_outputs
        
        # hidden layer errors = output_errors split by weights and recombined at hidden nodes
        hidden_errors = numpy.dot(self.w_ho.T, output_errors)
        
        # update the weights for the links beetween the hidden and output layers
        self.w_ho += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links beetween the input and hidden layers
        self.w_ih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        pass

    # query the neural network
    def query(self, inputs_list):
        
        # convert input list to 2-D array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate input and output signals for hidden layer
        hidden_inputs = numpy.dot(self.w_ih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate input and output signals for output layer
        final_inputs = numpy.dot(self.w_ho, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

    pass

# количество входных, скрытых и выходных узлов
input_nodes = 784
hidden_nodes = 100
output_nodes = 10

# коэффициент обучения
learning_rate = 0.3

# создаём экземпляр нейронной сети
n = NumbersRecognition(input_nodes, hidden_nodes, output_nodes, learning_rate)

# read data from file
def read_file(path_to_file):
    file = open(path_to_file, 'r')
    data = file.readlines()
    file.close()
    return data

# метод для масштабирования и смещения входных значений
def scale_input_values(values):
    # деление исходных входных значений, изменяющихся в диапазоне (0 - 255), на 255 приведёт их к диапазону (0 - 1,0)
    # последующее умножение этих значений на 0,99 приведёт их к диапазону (0,0 - 0,99)
    # далее мы инкрементируем их на 0,01, чтоб вместить их в желаемый диапазон (0,01 - 1,0)
    return (numpy.asfarray(values) / 255.0 * 0.99) + 0.01

# загружаем в список тренировочный набор данных
training_data_list = read_file("mnist_dataset/mnist_train.csv")
#training_data_list = read_file("mnist_dataset/mnist_train_100.csv")

# train neural network
for record in training_data_list:
    # получить список значений, используя символ "," в качестве разделителя
    all_training_values = record.split(',')
    # масштабировать и сместить входные значения
    scaled_inputs = scale_input_values(all_training_values[1:])
    # создать целевые выходные значения(все равны 0.01, за исключением маркерного значения)
    targets = numpy.zeros(output_nodes) + 0.01
    targets[int(all_training_values[0])] = 0.99
    
    n.train(scaled_inputs, targets)
    pass

# проверим роботу нейронки
test_data_list = read_file("mnist_dataset/mnist_test.csv")
#test_data_list = read_file("mnist_dataset/mnist_test_10.csv")

#all_test_values = test_data_list[0].split(',')
#print(all_test_values[0])

In [48]:
#image_array = numpy.asfarray(all_test_values[1:]).reshape((28, 28))
#matplotlib.pyplot.imshow(image_array, cmap = 'Greys', interpolation = 'None')

In [49]:
#n.query((numpy.asfarray(all_test_values[1:]) / 255.0 * 0.99) + 0.01)

In [75]:
# журнал оценок работы сети, первоначально пустой
scorecard = []

# перебрать все записи в тестовом наборе
for record in test_data_list:
    # получить список значений из записи, исполььзуя в качестве разделителя символ запятой
    all_values = record.split(',')
    # правильный ответ - первое значение
    correct_lable = int(all_values[0])
    #print(correct_lable, "Истинный маркер")
    # масштабировать и сместить входные значения
    inputs = scale_input_values(all_values[1:])
    # опрос сети
    outputs = n.query(inputs)
    # индекс наибольшего значения является маркерным значением
    label = numpy.argmax(outputs)
    #print(label, "Ответ сети")
    # в случае правильного ответа сети присоединить к списку значение 1, иначе - 0
    if (label == correct_lable):
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass
    pass


In [76]:
#print(scorecard)
scorecard_array = numpy.asarray(scorecard)
print("Эффективность нейронной сети -", scorecard_array.sum() / scorecard_array.size)

Эффективность нейронной сети - 0.9383
